In [ ]:
# !pip install aiohttp
import aiohttp
import asyncio
import os
import random
import time
import nest_asyncio

# Allow nested event loops (required for Jupyter Notebook)
nest_asyncio.apply()

# Server URL
URL = "http://192.168.1.11:8000/v1/yolo/detect/"

# Path to the image file
IMAGE_FOLDER = "./images/"

# Number of requests per second
REQUESTS_PER_SECOND = 50

# Timeout settings
TIMEOUT = aiohttp.ClientTimeout(total=60)  # 60 seconds timeout


def get_random_image_path(folder):
    """Get a random image path from the specified folder."""
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    if not files:
        raise FileNotFoundError("No files found in the specified folder.")
    return os.path.join(folder, random.choice(files))


async def send_request(session, url, image_path):
    """Send a single HTTP POST request with an image file."""
    try:
        with open(image_path, 'rb') as image_file:
            # Prepare multipart form-data
            form_data = aiohttp.FormData()
            form_data.add_field('file', 
                                image_file, 
                                filename=image_path.split('/')[-1], 
                                content_type='image/jpeg')
            
            # Send the POST request
            async with session.post(url, headers={"accept": "application/json"}, data=form_data) as response:
                print(f"Response: {response.status}")
                if response.status == 200:
                    result = await response.text()
                    print(f"Response Body: {result}")
                else:
                    print(f"Failed with status: {response.status}")
    except Exception as e:
        print(f"Request failed: {e}")

async def send_requests():
    """Send multiple requests to the server at a steady rate."""
    async with aiohttp.ClientSession(timeout=TIMEOUT) as session:
        while True:
            tasks = []
            for _ in range(REQUESTS_PER_SECOND):
                # Pick a random image from the folder
                random_image_path = get_random_image_path(IMAGE_FOLDER)
                tasks.append(send_request(session, URL, random_image_path))

            start_time = time.time()
            
            # Execute all tasks
            await asyncio.gather(*tasks)
            
            # Wait to maintain the desired rate
            elapsed = time.time() - start_time
            await asyncio.sleep(max(1 - elapsed, 0))

if __name__ == "__main__":
    try:
        asyncio.run(send_requests())
    except KeyboardInterrupt:
        print("Load testing stopped.")
